In [ ]:
import os
import pandas as pd
import pyodbc


server = 'juanlopezch.database.windows.net'
database = 'Spotify_charts'
username = '*********'
password = '*********'


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '***************************'

client_secret = '*************************'

user = '********'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Gathering unique songs uri from already created table.
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

sql_query = "select distinct uri from dbo.WeeklyCharts"

df = pd.read_sql(sql_query, cnxn)

cnxn.close()

In [ ]:
# Created a new DataFrame to storage processed data.
df_procesado = pd.DataFrame(columns=['uri', 'Cover', 'popularity', 'explicit', 'danceability',
                                    'energy', 'duration_ms'])

In [ ]:
# Iterate on every row
for index, row in result.iterrows():
    try:
        track_id = row['uri']
        track_info = sp.track(track_id)

        imagen_grande = track_info['album']['images'][0]['url']
        popularidad = track_info['popularity']
        explicita = track_info['explicit']

        temp_df = pd.DataFrame({'uri': [track_id], 'Cover': [imagen_grande], 
                                'popularity': [popularidad], 'explicit': [explicita]})

        df_procesado = pd.concat([df_procesado, temp_df], ignore_index=True)

        print(df_procesado.shape)
    except Exception as e:
        print(f"An error occurred in the iteration {index}: {e}")


In [ ]:
# Adding the data on a new table created on the Azure SQL database.
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
    
   
 #Insert Dataframe into SQL Server:
for index, row in df_procesado.iterrows():
     cursor.execute("INSERT INTO dbo.UniqueSongs (uri,Cover,Popularity,IsExplicit) values(?,?,?,?)", 
                    row['uri'], row['Cover'], row['popularity'], row['explicit'])
     cnxn.commit()


cursor.close()